# Downloading Meade speeches

In [5]:
import re
import bs4
import queue
import json
import sys
import csv
import urllib
import util
import os.path
import datetime
from os.path import join as pjoin
import time
import zipfile
import pandas as pd

In [6]:
website = "https://www.meade18.com/categorias/discursos/"

In [7]:
def build_soup(website):
    '''
    Takes a website and creates a soup.

    Inputs:
        website: a url

    Outputs:
        Soup object.
    '''

    sitio = util.get_request(website)
    siitio = sitio.text.encode('UTF-16')
    soup = bs4.BeautifulSoup(siitio, "html5lib")

    return soup

In [8]:
#Speech extractor
def extractor(url):
    speech = ""
    soup = build_soup(url)
    for p in soup.find_all('p'):
        speech += p.getText()
    return speech

In [9]:
#builds list of tuples (date,speech)
def soup_to_links(soup, link_list):


    for art in soup.find_all('article'):
        if art.attrs['role'] == "article":
            link = art.find('a')
            url = link.get('href')
            
            speech = extractor(url)
            
            time = art.find('time')
            date = time.get('datetime')[0:10]
        
            tup = [date, speech]
            link_list.append(tup)

    return(link_list)

In [10]:
#all the links with speeches
all_sites = [website] * 18
for i in range(1,18):
    num = i+1
    plug = str(num)
    all_sites[i] = all_sites[i]+"page/" + plug +"/"

In [11]:
data = []
for site in all_sites:
    soup = build_soup(site)
    links = soup_to_links(soup, data)

In [73]:
#Building and merging dataframes
meade_df = pd.DataFrame(data, columns = ['date', 'speech']) 
meade_df.to_csv('meade.csv',  encoding='UTF-16', index=False)